# Data Cleaning Challenge - Inconsistent data entry

## Intro

### All days of the challange:

* [Day 1: Handling missing values](./nb1-data-cleaning-challenge-handling-missing-values.ipynb)
* [Day 2: Scaling and normalization](./nb2-data-cleaning-challenge-scale-and-normalize-data.ipynb)
* [Day 3: Parsing dates](./nb3-data-cleaning-challenge-parsing-dates.ipynb)
* [Day 4: Character encodings](./nb4-data-cleaning-challenge-character-encodings.ipynb)
* [Day 5: Inconsistent Data Entry](./nb5-data-cleaning-challenge-inconsistent-data-entry.ipynb)
___

Welcome to day 5 of the 5-Day Data Challenge! (Can you believe it's already been five days??) Today, we're going to learn how to clean up inconsistent text entries. To get started, click the blue "Fork Notebook" button in the upper, right hand corner. This will create a private copy of this notebook that you can edit and play with. Once you're finished with the exercises, you can choose to make your notebook public to share with others. :)

> **Your turn!** As we work through this notebook, you'll see some notebook cells (a block of either code or text) that has "Your Turn!" written in it. These are exercises for you to do to help cement your understanding of the concepts we're talking about. Once you've written the code to answer a specific question, you can run the code by clicking inside the cell (box with code in it) with the code you want to run and then hit CTRL + ENTER (CMD + ENTER on a Mac). You can also click in a cell and then click on the right "play" arrow to the left of the code. If you want to run all the code in your notebook, you can use the double, "fast forward" arrows at the bottom of the notebook editor.

Here's what we're going to do today:

* [Get our environment set up](#Get-our-environment-set-up)
* [Do some preliminary text pre-processing](#Do-some-preliminary-text-pre-processing)
* [Use fuzzy matching to correct inconsistent data entry](#Use-fuzzy-matching-to-correct-inconsistent-data-entry)


Let's get started!

## Get our environment set up

The first thing we'll need to do is load in the libraries we'll be using. Not our datasets, though: we'll get to those later!

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

In [1]:
# modules we'll use
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# helpful modules
### fuzzywuzzy is deprecated ###
#import fuzzywuzzy
#from fuzzywuzzy import process
import thefuzz
from thefuzz import process
import chardet

# Handling directories
import os
import kaggle_cleaning
from kaggle_cleaning.config import RAW_DATA_DIR, CLEAN_DATA_DIR

# set seed for reproducibility
np.random.seed(0)

### Read CSV file

When I tried to read in the `PakistanSuicideAttacks Ver 11 (30-November-2017).csv`file the first time, I got a character encoding error, so I'm going to quickly check out what the encoding should be...

In [2]:
# look at the first ten thousand bytes to guess the character encoding
raw_pak = os.path.join(RAW_DATA_DIR, 'PakistanSuicideAttacks Ver 11 (30-November-2017).csv')
with open(raw_pak, 'rb') as rawdata:
    print(type(rawdata))
    result = chardet.detect(rawdata.read(100000))

# check what the character encoding might be
print(result)

<class '_io.BufferedReader'>
{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


And then read it in with the correct encoding. (If this look unfamiliar to you, check out [yesterday's challenge](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings/).) 

In [3]:
# read in our dat
suicide_attacks = pd.read_csv(raw_pak, 
                              encoding='Windows-1252')
suicide_attacks.head()

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaN,Islamabad,33.7180,73.0718,Capital,...,NaN,14.0,15.0,NaN,60,2.0,NaN,NaN,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,NaN,NaN,Karachi,24.9918,66.9911,Sindh,...,NaN,NaN,3.0,NaN,3,1.0,NaN,NaN,23.770,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,NaN,7:45 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,13.0,15.0,20.0,40,1.0,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
3,4,Friday-June 14-2002,3 Raby` al-THaany 1423 A.H,Working Day,NaN,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,NaN,12.0,NaN,51,1.0,NaN,NaN,31.430,88.574
4,5,Friday-July 4-2003,4 Jumaada al-awal 1424 A.H,Working Day,NaN,NaN,Quetta,30.2095,67.0182,Baluchistan,...,Shiite,44.0,47.0,NaN,65,1.0,NaN,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.120,91.616


Now we're ready to get started! You can, as always, take a moment here to look at the data and get familiar with it. :)

## Do some preliminary text pre-processing

For this exercise, I'm interested in cleaning up the "City" column to make sure there's no data entry inconsistencies in it. We could go through and check each row by hand, of course, and hand-correct inconsistencies when we find them. There's a more efficient way to do this though!

In [4]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

array(['ATTOCK', 'Attock ', 'Bajaur Agency', 'Bannu', 'Bhakkar ', 'Buner',
       'Chakwal ', 'Chaman', 'Charsadda', 'Charsadda ', 'D. I Khan',
       'D.G Khan', 'D.G Khan ', 'D.I Khan', 'D.I Khan ', 'Dara Adam Khel',
       'Dara Adam khel', 'Fateh Jang', 'Ghallanai, Mohmand Agency ',
       'Gujrat', 'Hangu', 'Haripur', 'Hayatabad', 'Islamabad',
       'Islamabad ', 'Jacobabad', 'KURRAM AGENCY', 'Karachi', 'Karachi ',
       'Karak', 'Khanewal', 'Khuzdar', 'Khyber Agency', 'Khyber Agency ',
       'Kohat', 'Kohat ', 'Kuram Agency ', 'Lahore', 'Lahore ',
       'Lakki Marwat', 'Lakki marwat', 'Lasbela', 'Lower Dir', 'MULTAN',
       'Malakand ', 'Mansehra', 'Mardan', 'Mohmand Agency',
       'Mohmand Agency ', 'Mohmand agency', 'Mosal Kor, Mohmand Agency',
       'Multan', 'Muzaffarabad', 'North Waziristan', 'North waziristan',
       'Nowshehra', 'Orakzai Agency', 'Peshawar', 'Peshawar ', 'Pishin',
       'Poonch', 'Quetta', 'Quetta ', 'Rawalpindi', 'Sargodha',
       'Sehwan town',

Just looking at this, I can see some problems due to inconsistent data entry: 'Lahore' and 'Lahore ', for example, or 'Lakki Marwat' and 'Lakki marwat'.

The first thing I'm going to do is make everything lower case (I can change it back at the end if I like) and remove any white spaces at the beginning and end of cells. Inconsistencies in capitalizations and trailing white spaces are very common in text data and you can fix a good 80% of your text data entry inconsistencies by doing this.

In [5]:
# convert to lower case
suicide_attacks['City'] = suicide_attacks['City'].str.lower()
# remove trailing white spaces
suicide_attacks['City'] = suicide_attacks['City'].str.strip()

In [6]:
suicide_attacks['City'].unique()

array(['islamabad', 'karachi', 'quetta', 'rawalpindi', 'north waziristan',
       'kohat', 'attock', 'sialkot', 'lahore', 'swat', 'hangu', 'bannu',
       'lasbela', 'malakand', 'peshawar', 'd.i khan', 'lakki marwat',
       'tank', 'gujrat', 'charsadda', 'kuram agency', 'shangla',
       'bajaur agency', 'south waziristan', 'haripur', 'sargodha',
       'nowshehra', 'mohmand agency', 'dara adam khel', 'khyber agency',
       'mardan', 'bhakkar', 'orakzai agency', 'buner', 'd.g khan',
       'pishin', 'chakwal', 'upper dir', 'muzaffarabad', 'totalai',
       'multan', 'lower dir', 'sudhanoti', 'poonch', 'mansehra', 'karak',
       'swabi', 'shikarpur', 'sukkur', 'chaman', 'd. i khan', 'khanewal',
       'fateh jang', 'taftan', 'tirah valley', 'wagah', 'zhob',
       'kurram agency', 'taunsa', 'jacobabad', 'shabqadar-charsadda',
       'khuzdar', 'ghallanai, mohmand agency', 'hayatabad',
       'mosal kor, mohmand agency', 'sehwan town',
       'tangi, charsadda district'], dtype=object

Next we're going to tackle more difficult inconsistencies.

In [7]:
# Your turn! Take a look at all the unique values in the "Province" column. 
provinces = suicide_attacks['Province'].unique()
provinces.sort()
provinces

array(['AJK', 'Balochistan', 'Baluchistan', 'Capital', 'FATA', 'Fata',
       'KPK', 'Punjab', 'Sindh'], dtype=object)

In [8]:
# Then convert the column to lowercase and remove any trailing white spaces
suicide_attacks['Province'] = suicide_attacks['Province'].str.lower()
suicide_attacks['Province'] = suicide_attacks['Province'].str.strip()
suicide_attacks['Province'].unique()

array(['capital', 'sindh', 'baluchistan', 'punjab', 'fata', 'kpk', 'ajk',
       'balochistan'], dtype=object)

## Use fuzzy matching to correct inconsistent data entry

Alright, let's take another look at the city column and see if there's any more data cleaning we need to do.

In [9]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd. i khan', 'd.g khan', 'd.i khan',
       'dara adam khel', 'fateh jang', 'ghallanai, mohmand agency',
       'gujrat', 'hangu', 'haripur', 'hayatabad', 'islamabad',
       'jacobabad', 'karachi', 'karak', 'khanewal', 'khuzdar',
       'khyber agency', 'kohat', 'kuram agency', 'kurram agency',
       'lahore', 'lakki marwat', 'lasbela', 'lower dir', 'malakand',
       'mansehra', 'mardan', 'mohmand agency',
       'mosal kor, mohmand agency', 'multan', 'muzaffarabad',
       'north waziristan', 'nowshehra', 'orakzai agency', 'peshawar',
       'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=

It does look like there are some remaining inconsistencies: 'd. i khan' and 'd.i khan' should probably be the same. (I [looked it up](https://en.wikipedia.org/wiki/List_of_most_populous_cities_in_Pakistan) and 'd.g khan' is a seperate city, so I shouldn't combine those.) 

I'm going to use the **[fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) (DEPRECATED**) (I, jgp-13, will use [thefuzz](https://github.com/seatgeek/thefuzz), an improved version of `fuzzywuzzy`) will use package to help identify which string are closest to each other. This dataset is small enough that we could probably could correct errors by hand, but that approach doesn't scale well. (Would you want to correct a thousand errors by hand? What about ten thousand? Automating things as early as possible is generally a good idea. Plus, it’s fun! :)

> **Fuzzy matching:** The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.

Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to get the ten strings from our list of cities that have the closest distance to "d.i khan".

In [10]:
# get the top 10 closest matches to "d.i khan"
matches = thefuzz.process.extract("d.i khan", cities, limit=10, scorer=thefuzz.fuzz.token_sort_ratio)

# take a look at them
matches

[('d. i khan', 100),
 ('d.i khan', 100),
 ('d.g khan', 88),
 ('khanewal', 50),
 ('sudhanoti', 47),
 ('hangu', 46),
 ('kohat', 46),
 ('dara adam khel', 45),
 ('chaman', 43),
 ('mardan', 43)]

We can see that two of the items in the cities are very close to "d.i khan": "d. i khan" and "d.i khan". We can also see the "d.g khan", which is a seperate city, has a ratio of 88. Since we don't want to replace "d.g khan" with "d.i khan", let's replace all rows in our City column that have a ratio of > 90 with "d. i khan". 

To do this, I'm going to write a function. (It's a good idea to write a general purpose function you can reuse if you think you might have to do a specific task more than once or twice. This keeps you from having to copy and paste code too often, which saves time and can help prevent mistakes.)

In [11]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = thefuzz.process.extract(string_to_match, strings, 
                                         limit=10, scorer=thefuzz.fuzz.token_sort_ratio)

    # only get matches with a ratio > min_ratio (default = 90)
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

Now that we have a function, we can put it to the test!

In [12]:
# use the function we just wrote to replace close matches to "d.i khan" with "d.i khan"
replace_matches_in_column(df=suicide_attacks, column='City', string_to_match="d.i khan")

All done!


And now let's can check the unique values in our City column again and make sure we've tidied up d.i khan correctly.

In [13]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'kurram agency', 'lahore', 'lakki marwat',
       'lasbela', 'lower dir', 'malakand', 'mansehra', 'mardan',
       'mohmand agency', 'mosal kor, mohmand agency', 'multan',
       'muzaffarabad', 'north waziristan', 'nowshehra', 'orakzai agency',
       'peshawar', 'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=object)

Excellent! Now we only have "d.i khan" in our dataframe and we didn't have to change anything by hand. 

In [14]:
# Your turn! It looks like 'Kuram Agency' and 'Kurram Agency' should
# be the same city. Correct the dataframe so that they are.
# After verifying online, 'Kurram Agency' is the correct name, and 'Kuram Agency' is a misspelling.

# Note: TheFuzz library has already been imported earlier.
import thefuzz 
from thefuzz import process


In [15]:
# get all the unique values in the 'province' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'kurram agency', 'lahore', 'lakki marwat',
       'lasbela', 'lower dir', 'malakand', 'mansehra', 'mardan',
       'mohmand agency', 'mosal kor, mohmand agency', 'multan',
       'muzaffarabad', 'north waziristan', 'nowshehra', 'orakzai agency',
       'peshawar', 'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=object)

In [16]:
matches = process.extract('kurram agency', cities, 
                                         limit=10, scorer=thefuzz.fuzz.token_sort_ratio)
matches

[('kurram agency', 100),
 ('kuram agency', 96),
 ('bajaur agency', 69),
 ('khyber agency', 69),
 ('orakzai agency', 67),
 ('mosal kor, mohmand agency', 59),
 ('mohmand agency', 59),
 ('ghallanai, mohmand agency', 49),
 ('gujrat', 42),
 ('d.g khan', 38)]

In [17]:
# only get matches with a ratio > 95
close_matches = [matches[0] for matches in matches if matches[1] >= 95]
close_matches

['kurram agency', 'kuram agency']

In [18]:
# get the rows of all the close matches in our dataframe
rows_with_matches = suicide_attacks['City'].isin(close_matches)
suicide_attacks[suicide_attacks['City']=='kuram agency']

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
53,54,Saturday-August 4-2007,21 Rajab 1428 A.H,Working Day,NaN,10:30am,kuram agency,32.9746,70.1456,fata,...,Sunni,9.0,23.0,35.0,43,1.0,NaN,1.Agency Headqurters \nHospital,25.835,78.503
94,95,Saturday-February 16-2008,9 safar 1429 A.H.,Working Day,NaN,evening,kuram agency,35.2227,72.4258,fata,...,NaN,38.0,47.0,109.0,110,1.0,NaN,Agency headquarter hospital,12.725,54.905
341,342,Friday-February 17-2012,NaN,Working Day,NaN,2:00 PM,kuram agency,32.9746,70.1456,fata,...,Shiite,21.0,28.0,NaN,36,1.0,NaN,Agency Headquarter Hospital,0.040,32.072
355,356,Monday-September 10-2012,NaN,Working Day,NaN,2:40pm,kuram agency,32.9746,70.1456,fata,...,NaN,14.0,15.0,40.0,80,1.0,NaN,NaN,19.935,67.883
396,397,Friday-July 26-2013,NaN,Working Day,NaN,Evening,kuram agency,32.9746,70.1456,kpk,...,Shiite,43.0,50.0,180.0,200,2.0,NaN,Parachinar Headquarters Hospital,26.015,78.827


In [19]:
# replace all rows with close matches with the input matches 
suicide_attacks.loc[rows_with_matches, 'City'] = 'kurram agency'

suicide_attacks[suicide_attacks['City']=='kuram agency']

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)


And that's it for today! If you have any questions, be sure to post them in the comments below or [on the forums](https://www.kaggle.com/questions-and-answers). 

Remember that your notebook is private by default, and in order to share it with other people or ask for help with it, you'll need to make it public. First, you'll need to save a version of your notebook that shows your current work by hitting the "Commit & Run" button. (Your work is saved automatically, but versioning your work lets you go back and look at what it was like at the point you saved it. It also lets you share a nice compiled notebook instead of just the raw code.) Then, once your notebook is finished running, you can go to the Settings tab in the panel to the left (you may have to expand it by hitting the [<] button next to the "Commit & Run" button) and setting the "Visibility" dropdown to "Public".

## More practice!

Do any other columns in this dataframe have inconsistent data entry? If you can find any, try to tidy them up.

You can also try reading in the `PakistanSuicideAttacks Ver 6 (10-October-2017).csv` file from this dataset and tidying up any inconsistent columns in that data file.

In [20]:
psa_file_v6 = os.path.join(RAW_DATA_DIR, 'PakistanSuicideAttacks Ver 6 (10-October-2017).csv')
with open(psa_file_v6, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(20000))
result['encoding']

'Windows-1252'

In [21]:
df = pd.read_csv(psa_file_v6, encoding = result['encoding'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   S#                       492 non-null    int64  
 1   Date                     492 non-null    object 
 2   Islamic Date             336 non-null    object 
 3   Blast Day Type           481 non-null    object 
 4   Holiday Type             72 non-null     object 
 5   Time                     281 non-null    object 
 6   City                     492 non-null    object 
 7   Latitude                 490 non-null    float64
 8   Longitude                490 non-null    object 
 9   Province                 492 non-null    object 
 10  Location                 489 non-null    object 
 11  Location Category        457 non-null    object 
 12  Location Sensitivity     456 non-null    object 
 13  Open/Closed Space        457 non-null    object 
 14  Influencing Event/Event  1

In [22]:
df.describe()

,S#,Latitude,Killed Min,Killed Max,Injured Min,No. of Suicide Blasts,Temperature(C),Temperature(F)
count,492.000000,490.000000,346.000000,476.00000,361.000000,410.000000,487.000000,485.000000
mean,246.500000,32.621874,14.855491,15.30042,31.601108,1.117073,21.075554,69.906373
std,142.172431,2.477845,17.659154,20.32867,38.762832,0.396749,8.390859,15.108953
min,1.000000,24.879503,0.000000,0.00000,0.000000,1.000000,-2.370000,27.734000
25%,123.750000,31.823800,3.000000,3.00000,7.000000,1.000000,14.617500,58.253000
50%,246.500000,33.583300,8.000000,8.00000,20.000000,1.000000,21.295000,70.331000
75%,369.250000,34.004300,20.750000,19.00000,40.000000,1.000000,28.115000,82.499000
max,492.000000,35.383300,125.000000,148.00000,320.000000,4.000000,44.000000,111.000000


In [23]:
df.describe(include= ['category','object'])

,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Longitude,Province,Location,Location Category,Location Sensitivity,Open/Closed Space,Influencing Event/Event,Target Type,Targeted Sect if any,Injured Max,Explosive Weight (max),Hospital Names
count,492,336,481,72,281,492,490,492,489,457,456,457,187,466,97,460,169,294
unique,451,301,3,15,216,93,97,9,483,24,4,6,169,24,7,91,112,236
top,Thursday-July 19-2007,5 Rajab 1428 A.H,Working Day,Weekend,Evening,Peshawar,71.5448,KPK,Imambargah,Police,High,Open,Sectarian strife,Military,Shiite,2,6kg,Lady Reading Hospital
freq,3,3,398,45,12,71,71,250,3,90,264,317,7,115,38,20,8,20


### Observations from the Preliminary Analysis of the Dataset

Following an initial review of the dataset, the following adjustments and refinements are recommended:

- The `'Date'` column should be converted to `datetime` format.
- The `'Longitude'` column should be converted to `float` type.
- There are two rows where both `'Latitude'` and `'Longitude'` contain null values. If a city and/or province is recorded, the geographical location can be researched and imputed accordingly.
- The `'Injured Max'` column should be converted to a numeric type.
- The following columns should be converted to `integer` type: `'Killed min'`, `'Killed max'`, `'Injured min'`, `'Injured max'`, and `'Suicide Blasts'`.
- The `'Explosive Weight (max)'` column may require further analysis to ensure accuracy.
- The suitability of converting temperatures from Celsius to Fahrenheit (`Temperature C to F`) should be assessed.
- The `"Open/Closed Space"` column contains six unique values, whereas the expected number was only two or possibly three. This discrepancy suggests inconsistencies that require further investigation and standardisation.
- Lists such as cities and provinces should be reviewed for consistency and correctness.

### Standardising Date Formats and Correcting Misspellings

This cell addresses inconsistencies in the `'Date'` column by performing the following steps:

- Corrects a known misspelling in the dataset (`'Thursay' → 'Thursday'`).
- Defines a list of expected date formats to ensure proper conversion.
- Implements a function to iteratively attempt conversion using the predefined formats.
- Applies the function to standardise all date entries.
- Identifies any remaining entries that could not be converted, allowing for further investigation.


In [24]:
# Fix incorrect mispelled date value
df.loc[452, 'Date'] = 'Thursday-Aug 27-2015'

# List of possible date formats
date_formats = [
    '%A-%B %d-%Y',
    '%A-%B-%d-%Y',
    '%A-%b-%d-%Y',    
    '%A-%b %d-%Y'     
]

# Function to convert dates by trying multiple formats
def parse_dates(date):
    for fmt in date_formats:
        try:
            return pd.to_datetime(date, format=fmt)
        except ValueError:
            continue
    return pd.NaT  # Return NaT if no format matches

# Convert and replace the 'Date' column
df['Date'] = df['Date'].apply(parse_dates)

# Check for any remaining NaT values
df[df['Date'].isna()]


,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)


### Converting Longitude to Float and Handling Missing Values

This cell ensures the `'Longitude'` column is correctly formatted as a numeric type and addresses missing values in the dataset:

- Converts the `'Longitude'` column to `float` for consistency in numerical operations.
- Identifies and displays rows where `'Longitude'` values are missing.
- Since both missing entries have corresponding `'City'` and `'Province'` values, their geographic coordinates are retrieved from **reliable sources**.
- The missing latitude and longitude values are manually updated using coordinates from Wikipedia (last checked on **9th March 2025**):
  - **Lahore, Punjab** → (`Latitude: 31.549722, Longitude: 74.343611`)
  - **Quetta, Balochistan** → (`Latitude: 30.183333, Longitude: 67.000000`)
- The corrected entries are displayed for verification.

In [25]:
# Lets change the type of the Longitude columnt to float
df['Longitude'] = df['Longitude'].astype(float)

# Then Lets check the positions where there are two empty values
display(df[df['Longitude'].isna()])

# As there is a City and Province for the two empty values lets get it from Internet (both last checked on 2025-03-09)
# https://en.wikipedia.org/wiki/Lahore 
df.loc[449, ['Latitude','Longitude']]=[31.549722, 74.343611] 
# https://en.wikipedia.org/wiki/Quetta 
df.loc[450, ['Latitude','Longitude']]=[30.183333, 67.0] 
display(df.loc[[449,450],:])

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
449,450,2015-06-29,NaN,Working Day,NaN,NaN,Lahore,NaN,NaN,Punjab,...,NaN,1.0,4.0,NaN,http://www.samaa.tv/pakistan/29-Jun-2015/shoot...,NaN,NaN,NaN,NaN,NaN
450,451,2015-07-17,NaN,Holiday,NaN,NaN,Quetta,NaN,NaN,Balochistan,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
449,450,2015-06-29,NaN,Working Day,NaN,NaN,Lahore,31.549722,74.343611,Punjab,...,NaN,1.0,4.0,NaN,http://www.samaa.tv/pakistan/29-Jun-2015/shoot...,NaN,NaN,NaN,NaN,NaN
450,451,2015-07-17,NaN,Holiday,NaN,NaN,Quetta,30.183333,67.000000,Balochistan,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Correcting  `'Injured Max'` column

This cell processes the `'Injured Max'` column to maintain numerical consistency:

- Converts `'Injured Max'` to numeric values, handling non-numeric entries as `NaN`.
- Fills missing values in `'Injured Max'` with the column’s median to preserve distribution integrity.
- Ensures that `'Injured Max'` is at least equal to `'Injured Min'` by applying a row-wise maximum.

This guarantees that the reported maximum injuries are never lower than the minimum injuries.

In [26]:
# Convert 'Injured Max' and 'Injured Min' to numeric, converting non-numeric values to NaN
df['Injured Max'] = pd.to_numeric(df['Injured Max'], errors='coerce')

# Fill missing values in 'Injured Max' with the median
df['Injured Max'] = df['Injured Max'].fillna(df['Injured Max'].median())

# Ensure 'Injured Max' is at least equal to 'Injured Min'
df['Injured Max'] = df[['Injured Max', 'Injured Min']].max(axis=1)


### Cleaning and Standardising the 'Open/Closed Space' Column

This cell ensures consistency in the `'Open/Closed Space'` column by performing the following steps:

- **Retrieves unique values** to identify inconsistencies in formatting.
- **Standardises text formatting** by:
  - Removing leading and trailing spaces.
  - Converting all values to lowercase to ensure uniformity.
- **Replaces ambiguous entries**:
  - The category `'open/closed'` is converted to `NaN` to reflect missing or unclear data.
- **Displays the updated value counts**, including `NaN` values, to verify the cleaning process.


In [27]:
df["Open/Closed Space"].unique()

array(['Closed', 'Open', 'open', 'Open ', nan, 'closed', 'Open/Closed'],
      dtype=object)

In [28]:
df["Open/Closed Space"] = df["Open/Closed Space"].str.strip().str.lower()
df["Open/Closed Space"] = df["Open/Closed Space"].replace({"open/closed":np.nan})
df["Open/Closed Space"].value_counts(dropna=False)

Open/Closed Space
open      329
closed    127
NaN        36
Name: count, dtype: int64

### Checking and Filling Missing Temperature Data

This step ensures consistency in temperature values by identifying and addressing missing data:

- **Identifies rows where either `Temperature(C)` or `Temperature(F)` is missing** and displays them for review.
- **Uses the conversion function `C2F()`** to fill missing Fahrenheit values where the corresponding Celsius value is available.
- **Updates specific missing Fahrenheit values at index `475` and `490`**, ensuring accuracy while preserving the original dataset.
- **Leaves rows where both temperature values are missing untouched**, as further work can be done to estimate these values using average temperatures for the corresponding zone.

This approach ensures that temperature data remains complete where possible while highlighting areas that require additional imputation.

In [29]:
def C2F(celsius):
    return (celsius * 9/5) + 32
def F2C(fahrenheit):
    return (fahrenheit - 32) * 5/9

In [30]:
print(df[(df['Temperature(C)'].isna())| (df['Temperature(F)'].isna())][['Temperature(C)','Temperature(F)']])

     Temperature(C)  Temperature(F)
165             NaN             NaN
449             NaN             NaN
450             NaN             NaN
473             NaN             NaN
475            19.0             NaN
479             NaN             NaN
490            29.0             NaN


In [31]:
df.loc[475, 'Temperature(F)'] = C2F(df.loc[475, 'Temperature(C)'])
df.loc[490, 'Temperature(F)'] = C2F(df.loc[490, 'Temperature(C)'])

### Cleaning Cities and Provinces

#### Standardising City Names for Consistency

This cell ensures that city names in the dataset are formatted uniformly by applying the following transformations:

- **Converts all city names to lowercase** to maintain consistency and avoid case-sensitive mismatches.
- **Removes any leading or trailing whitespace** to eliminate unintentional variations.
- **Extracts all unique city names** from the dataset to identify inconsistencies.
- **Sorts the unique city names alphabetically** to facilitate easier review and verification.

These steps help standardise the data, making it easier to detect duplicates, match locations accurately, and perform further cleaning if necessary.


In [32]:
# convert to lower case
df['City'] = df['City'].str.lower()
# remove trailing white spaces
df['City'] = df['City'].str.strip()

# get all the unique values in the 'province' column
cities = df['City'].unique()
# sort them alphabetically and then take a closer look
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd. i khan', 'd.g khan', 'd.i khan',
       'dara adam khel', 'fateh jang', 'ghallanai, mohmand agency',
       'gujrat', 'hangu', 'haripur', 'hayatabad', 'islamabad',
       'jacobabad', 'karachi', 'karak', 'khanewal', 'khuzdar',
       'khyber agency', 'kohat', 'kuram agency', 'kurram agency',
       'lahore', 'lakki marwat', 'lasbela', 'lower dir', 'malakand',
       'mansehra', 'mardan', 'mohmand agency',
       'mosal kor, mohmand agency', 'multan', 'muzaffarabad',
       'north waziristan', 'nowshehra', 'orakzai agency', 'peshawar',
       'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=

This cell ensures that city names are corrected and standardised by using fuzzy string matching to identify and replace similar variations. The process follows these steps:

- **Defines a set of standard city names** (`d.i khan`, `kurram agency`, `mohmand agency`, `charsadda`) to be used as reference.
- **Uses fuzzy matching (`thefuzz.process.extract`)** to find the closest matches for each standard name within the dataset.
- **Filters matches with a similarity score of 90 or higher**, ensuring only highly similar names are considered.
- **Replaces all identified variations with the corresponding standard name**, ensuring consistency in city names.

By applying this approach, variations in spelling, formatting, or abbreviations are resolved, making the dataset more uniform and easier to analyse.


In [33]:
standard_names = ["d.i khan", "kurram agency", "mohmand agency", "charsadda"]

for name in standard_names:
    # Find the closest matches with a score of 90 or higher
    matches = process.extract(name, cities, limit=5)
    close_matches = [match[0] for match in matches if match[1] >= 90]
    
    print(f"{name}: {close_matches}")  # Display detected similar names
    
    # Replace all close matches with the standard name
    df.loc[df['City'].isin(close_matches), 'City'] = name


d.i khan: ['d.i khan', 'd. i khan']
kurram agency: ['kurram agency', 'kuram agency']
mohmand agency: ['mohmand agency', 'ghallanai, mohmand agency', 'mosal kor, mohmand agency']
charsadda: ['charsadda', 'shabqadar-charsadda', 'tangi, charsadda district']


#### Standardising Province Names

A similar procedure is applied to ensure consistency in province names within the dataset. This involves:

- **Converting all names to lowercase** and **removing trailing spaces** to eliminate formatting inconsistencies.
- **Identifying and replacing variations** (e.g., alternate spellings) to align with a standard set of province names.
- **Ensuring uniformity** across the dataset, making it more reliable for analysis.

This process helps prevent mismatches and inconsistencies that could affect grouping, filtering, and interpretation of the data.


In [34]:
# convert to lower case
df['Province'] = df['Province'].str.lower()
# remove trailing white spaces
df['Province'] = df['Province'].str.strip()

# get all the unique values in the 'province' column
provinces = df['Province'].unique()
# sort them alphabetically and then take a closer look
provinces.sort()
provinces

array(['ajk', 'balochistan', 'baluchistan', 'capital', 'fata', 'kpk',
       'punjab', 'sindh'], dtype=object)

In [35]:
df["Province"] = df["Province"].replace({"baluchistan": "balochistan", 'capital':'ict', 'fata':'kpk'})

In [36]:
print(df['Province'].unique())

['ict' 'sindh' 'balochistan' 'punjab' 'kpk' 'ajk']


In [37]:
print(df['City'].unique())

['islamabad' 'karachi' 'quetta' 'rawalpindi' 'north waziristan' 'kohat'
 'attock' 'sialkot' 'lahore' 'swat' 'hangu' 'bannu' 'lasbela' 'malakand'
 'peshawar' 'd.i khan' 'lakki marwat' 'tank' 'gujrat' 'charsadda'
 'kurram agency' 'shangla' 'bajaur agency' 'south waziristan' 'haripur'
 'sargodha' 'nowshehra' 'mohmand agency' 'dara adam khel' 'khyber agency'
 'mardan' 'bhakkar' 'orakzai agency' 'buner' 'd.g khan' 'pishin' 'chakwal'
 'upper dir' 'muzaffarabad' 'totalai' 'multan' 'lower dir' 'sudhanoti'
 'poonch' 'mansehra' 'karak' 'swabi' 'shikarpur' 'sukkur' 'chaman'
 'khanewal' 'fateh jang' 'taftan' 'tirah valley' 'wagah' 'zhob' 'taunsa'
 'jacobabad' 'khuzdar' 'hayatabad' 'sehwan town']


In [39]:
pakistan_csv_file=os.path.join(CLEAN_DATA_DIR, 'pakistan_clean.csv')
df.to_csv(pakistan_csv_file)